In [ ]:
# Required libs
!pip3 install svgpathtools opencv-python numpy aggdraw

In [8]:
def to_tuple(compl):
    return (int(compl.real * 3), int(compl.imag * 3))

In [9]:
import math

def angle_between_vectors(vector_1, vector_2):
    # Each vector is an array [x, y] for its coordinates
    
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    return np.arccos(dot_product) / math.pi * 360

In [10]:
def two_points_dist(a, b):
    return (abs(a[0] - b[0]) ** 2 + abs(a[1] - b[1]) ** 2) ** 0.5

In [27]:
def curve_steepness(p0, p1, p2, p3):
    # Here, curve starts in p0 and ends in p3. Where p1, p2 are `control points` for curve (p1 for p0 and p2 for p3 respectively) (each point is a object of class Point)
    
    chord = (p3 - p0).length()
    cont_net = (p0 - p1).length() + (p2 - p1).length() + (p3 - p2).length()
    curve_length = (cont_net + chord) / 2;  # This is just an approximation
    
    vector_1 = [p1.x - p0.x, p1.y - p0.y]
    vector_2 = [p3.x - p2.x, p3.y - p2.y]
    
#     print(angle_between_vectors(vector_1, vector_2), vector_1, vector_2)
    return curve_length / angle_between_vectors(vector_1, vector_2)

In [32]:
# In DFS we are merging beizer curves into strokes
def dfs_on_segments(v, color, segm, usd, groups):
    # Choose where to go from curve
    
    usd[v] = True
    groups[v] = color
    current = segm[v]
    
    for u, other in enumerate(segm):
        
        if not usd[u]:
            steepness = curve_steepness(current[1], current[0], other[0], other[1])
            steepness = max(steepness, curve_steepness(current[1], current[0], other[3], other[2]))
            steepness = max(steepness, curve_steepness(current[2], current[3], other[0], other[1]))
            steepness = max(steepness, curve_steepness(current[2], current[3], other[3], other[2]))
            
            if (steepness > 1000):
                dfs_on_segments(u, color, segm, usd, groups)


In [33]:
import cv2
import numpy as np
from random import randint

from PIL import Image
import aggdraw

from svgpathtools import svg2paths, wsvg, svg2paths2


class Point:
    def __init__(self, i, j):
        self.real = i
        self.imag = j
        
        # TODO: use only x and y
        self.x = i
        self.y = j

    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)

    def length(self):
        return (self.x * self.x + self.y * self.y) # Not taking square root for optimisation purposes



ANGLE_EPS = 6.28 / 5
MAX_STEEPNESS = 10
IMAGE_SIZE_MULTIPLIER = 1.7

block_size = 366

segments = [[[] for i in range(6)] for j in range(6)]

paths, attributes, svg_attributes = svg2paths2(f'svg/{5}.svg')


img = np.zeros((6, 6, 1000, 1000, 3))


for path in paths:
    for curve in path:  # Path contains all curves in image        
        i, j = int(curve.start.real // block_size), int(curve.start.imag // block_size)
        p0 = Point(curve.start.real - block_size * i, curve.start.imag - block_size * j)
        p3 = Point(curve.end.real - block_size * i, curve.end.imag - block_size * j)
        
        if (type(curve).__name__ == "CubicBezier"):
            # Two more points for curve
            p1 = Point(curve.control1.real - block_size * i, curve.control1.imag - block_size * j)
            p2 = Point(curve.control2.real - block_size * i, curve.control2.imag - block_size * j)

            img[i, j] = cv2.line(img[i, j], to_tuple(p0), to_tuple(p1), (255, 0, 0), 1)
            img[i, j] = cv2.line(img[i, j], to_tuple(p2), to_tuple(p3), (0, 255, 0), 1)
            
            steepness = curve_steepness(p0, p1, p2, p3)
            
            if (steepness > MAX_STEEPNESS):
                segments[i][j].append([p0, p1, p2, p3])

        elif (type(curve).__name__ == "Line"):
            img[i, j] = cv2.line(img[i, j], to_tuple(p0), to_tuple(p3), (255, 0, 0), 1)
            segments[i][j].append([p0, p0, p3, p3])

    
    debug_colors = []  
    for i in range(6):
        for j in range(6):
            if len(segments[i][j]) == 0:
                continue
            
            name = str(j * 6 + i).zfill(2)
            
            cl = 0
            used = [False] * len(segments[i][j])
            groups = [0] * len(segments[i][j])
            
            # Run `dfs` on curves
            for ind, elem in enumerate(segments[i][j]):
                if not used[ind]:
                    dfs_on_segments(ind, cl, segments[i][j], used, groups)
                    cl += 1
            
#             print()
#             print(len(segments[i][j]))
#             print()
            
            # Assign random colours to each curve
            debug_color = [(0, 0, 0)] # Where index is the number of group (strike). Can be found in groups[i] from curve number
            for _ in range(1, max(groups) + 1):
                debug_color.append((randint(0, 255), randint(0, 255), randint(0, 255)))
         
            # Setup
            curves = Image.new("RGBA", (500, 500)) # last part is image dimensions
            draw = aggdraw.Draw(curves)
            fill = aggdraw.Brush("transparent")
            
            # Draw all selected curves
            print(groups)
            for ind, segm in enumerate(segments[i][j]):
                # outline = aggdraw.Pen((randint(0, 255), randint(0, 255), randint(0, 255)), 4)  # Random color (for debug)
                outline = aggdraw.Pen(debug_color[groups[ind]], 4)
                
                pathstring = f"m{segm[0].x * IMAGE_SIZE_MULTIPLIER} {segm[0].y * IMAGE_SIZE_MULTIPLIER} C{segm[1].x * IMAGE_SIZE_MULTIPLIER} {segm[1].y * IMAGE_SIZE_MULTIPLIER},{segm[2].x * IMAGE_SIZE_MULTIPLIER} {segm[2].y * IMAGE_SIZE_MULTIPLIER},{segm[3].x * IMAGE_SIZE_MULTIPLIER} {segm[3].y * IMAGE_SIZE_MULTIPLIER}"
                
                symbol = aggdraw.Symbol(pathstring)
                draw.symbol((0, 0), symbol, outline)

#                 draw.line((segm[0].x, segm[0].y, segm[3].x, segm[3].y), outline)  # Debug line from curve start to end
                draw.flush()
    
            
            smb_num = 5 * 36 + j * 6 + i
#             print(smb_num)
#             print(f'res/{smb_num // 48 + 1}/{smb_num % 48}_th.png')
            
            org = cv2.imread(f'res/{smb_num // 48 + 1}/{smb_num % 48}_th.png')
            cv2.imwrite(f"curves/curves{name}_org.png", org)
            
            curves.save(f"curves/curves{name}.png")

/tmp/ipykernel_9635/3935083933.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  return curve_length / angle_between_vectors(vector_1, vector_2)
/tmp/ipykernel_9635/437642093.py:7: RuntimeWarning: invalid value encountered in true_divide
  unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
/tmp/ipykernel_9635/437642093.py:6: RuntimeWarning: invalid value encountered in true_divide
  unit_vector_1 = vector_1 / np.linalg.norm(vector_1)


[0, 0, 1, 2, 3, 4, 4, 0, 5]
[0, 1, 1, 2, 2, 3, 4]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 1, 2, 1]
[0, 1, 2, 3, 4, 2, 5, 6, 3, 3]
[0, 0, 1, 2, 2, 2, 2, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 1, 1, 0, 2]
[0, 1, 2, 2, 3, 4]
[0, 1, 2, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 1]
[0, 0, 1, 0, 2, 3, 3, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2]
[0, 1, 2, 2, 2, 2, 3, 4]
[0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 2, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 2, 2, 3, 4, 1]
[0, 1, 2, 2, 3, 2]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 3, 4, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 2]
[0, 1, 2, 3, 3, 3]
[0, 1, 2, 2, 3, 2, 4]
[0, 0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Sandbox zone below

In [ ]:
# Alternative library for SVG parsing

from xml.dom import minidom

doc = minidom.parse("input.svg")
path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
doc.unlink()

In [ ]:
# Test .svg draw

img = Image.new("RGBA", (1000,1000)) # last part is image dimensions
draw = aggdraw.Draw(img)
outline = aggdraw.Pen("black", 5) # 5 is the outlinewidth in pixels
fill = aggdraw.Brush("yellow")

# the pathstring:
# m for starting point
# c for bezier curves
# z for closing up the path, optional
# (all lowercase letters for relative path)
pathstring = " c300,300,700,600,300,900 "

# create symbol
symbol = aggdraw.Symbol(pathstring)

xy = (20,20) # xy position to place symbol
draw.symbol(xy, symbol, outline, fill)
draw.flush()

pathstring = " c400,400,800,700,400,1000 "

# create symbol
symbol = aggdraw.Symbol(pathstring)

# draw and save it
xy = (20,20) # xy position to place symbol
draw.symbol(xy, symbol, outline, fill)
draw.flush()
img.save("testbeziercurves.png") # this image gets saved to same folder as the script